# Preflop Shove Explorer

Investigate preflop all-in ranges by action depth (open shoves, 3-bet shoves, and deeper).


In [1]:

from pathlib import Path
import os

def _locate_project_root() -> Path:
    current = Path().resolve()
    for candidate in (current, *current.parents):
        if (candidate / 'AGENTS.md').exists():
            return candidate
    raise FileNotFoundError('Repository root not found from notebook location.')

PROJECT_ROOT = _locate_project_root()
del _locate_project_root


In [2]:

import sys
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

DB_CANDIDATES = [
    Path(r'T:\Dev\ignition\drivehud\drivehud.db'),
    Path('/mnt/t/Dev/ignition/drivehud/drivehud.db'),
    PROJECT_ROOT / 'drivehud' / 'drivehud.db',
]

for candidate in DB_CANDIDATES:
    if candidate.exists():
        DB_PATH = candidate
        break
else:
    checked = os.linesep.join(str(p) for p in DB_CANDIDATES)
    message = 'Database not found. Checked:' + os.linesep + checked
    raise FileNotFoundError(message)

CACHE_PATH = PROJECT_ROOT / 'analysis' / 'cache' / 'preflop_shove_events.json'
FORCE_RELOAD = False


In [3]:

import json
import sqlite3
import xml.etree.ElementTree as ET

import numpy as np
import math
import pandas as pd
from IPython.display import display, HTML
from matplotlib.colors import LinearSegmentedColormap, TwoSlopeNorm

from analysis.sqlite_utils import connect_readonly
from analysis.cbet_utils import (
    BET_TYPES,
    RAISE_TYPES,
    parse_cards_text,
    extract_big_blind,
)


In [4]:

RANKS = ['A', 'K', 'Q', 'J', 'T', '9', '8', '7', '6', '5', '4', '3', '2']
RANK_INDEX = {rank: idx for idx, rank in enumerate(RANKS)}
CATEGORY_LABELS = {
    1: 'First to Bet Shove',
    2: '3-Bet Shove',
    3: '4-Bet Shove',
    4: '5+ Bet Shove',
}
CATEGORY_ORDER = [
    'First to Bet Shove',
    '3-Bet Shove',
    '4-Bet Shove',
    '5+ Bet Shove',
]
BLUE_CMAP = LinearSegmentedColormap.from_list('shove_blue', ['#ffffff', '#1d4ed8'])
HAND_GROUPS_ORDER = [
    'AA',
    'KK',
    'QQ',
    'JJ',
    'TT',
    'Other Pair',
    'AK',
    'ATs - AQs',
    'A2s - A9s',
    'ATo - AQo',
    'A2o - A9o',
    'Other Broadway Pair',
    'Other',
]
SUMMARY2_GROUPS_ORDER = [
    'KK - AA',
    'TT - QQ',
    '22 - 99',
    'AK',
    'Any Other Ace',
    'All Others',
]
BROADWAY_RANKS = {'A', 'K', 'Q', 'J', 'T'}
BROADWAY_COMBOS = {
    frozenset({'K', 'Q'}),
    frozenset({'K', 'J'}),
    frozenset({'K', 'T'}),
    frozenset({'Q', 'J'}),
    frozenset({'Q', 'T'}),
    frozenset({'J', 'T'}),
}


In [5]:

def _categorise_shove(level: int) -> str | None:
    if level <= 0:
        return None
    if level == 1:
        return CATEGORY_LABELS[1]
    if level == 2:
        return CATEGORY_LABELS[2]
    if level == 3:
        return CATEGORY_LABELS[3]
    return CATEGORY_LABELS[4]


def _is_all_in_action(action) -> bool:
    act_type = action.attrib.get('type')
    if act_type == '7':
        return True
    return action.attrib.get('allin', '').lower() in {'1', 'true', 'yes'}


def load_preflop_shove_events(db_path: Path, cache_path: Path | None = None, force: bool = False):
    if cache_path and cache_path.exists() and not force:
        with cache_path.open('r', encoding='utf-8') as fh:
            cached = json.load(fh)
        if cached:
            return cached

    events: list[dict] = []
    with connect_readonly(db_path) as conn:
        conn.row_factory = sqlite3.Row
        cur = conn.cursor()
        cur.execute('SELECT HandHistoryId, HandNumber, HandHistory FROM HandHistories')
        for row in cur:
            try:
                root = ET.fromstring(row['HandHistory'])
            except ET.ParseError:
                continue

            big_blind = extract_big_blind(root)
            if big_blind is None or big_blind <= 0:
                continue

            pocket_cards: dict[str, list] = {}
            for node in root.findall('.//round[@no="1"]/cards'):
                player = node.attrib.get('player')
                cards = parse_cards_text(node.text)
                if player and len(cards) == 2:
                    pocket_cards[player] = cards
            if not pocket_cards:
                continue

            aggressive_level = 0
            total_pot = 0.0

            preflop_actions = []
            for rnd in root.findall('.//round'):
                if rnd.attrib.get('no') == '1':
                    preflop_actions.extend(rnd.findall('action'))
            if not preflop_actions:
                continue

            for action in preflop_actions:
                player = action.attrib.get('player')
                if not player:
                    continue
                act_type = action.attrib.get('type')
                amount_text = action.attrib.get('sum') or action.attrib.get('bet') or '0'
                try:
                    amount = float(amount_text)
                except ValueError:
                    amount = 0.0

                if amount > 0:
                    total_pot += amount

                if act_type not in BET_TYPES.union(RAISE_TYPES):
                    continue
                if amount <= 0:
                    continue

                aggressive_level += 1
                if not _is_all_in_action(action):
                    continue

                category = _categorise_shove(aggressive_level)
                if category is None:
                    continue
                hero_cards = pocket_cards.get(player)
                if not hero_cards:
                    continue
                hole_cards = ' '.join(card for _, _, card in hero_cards)
                pot_before = total_pot - amount
                events.append({
                    'hand_number': row['HandNumber'],
                    'player': player,
                    'category': category,
                    'aggressive_level': aggressive_level,
                    'hole_cards': hole_cards,
                    'bet_amount': amount,
                    'bet_amount_bb': amount / big_blind if big_blind else None,
                    'pot_before': pot_before,
                    'big_blind': big_blind,
                })

    if cache_path:
        cache_path.parent.mkdir(parents=True, exist_ok=True)
        with cache_path.open('w', encoding='utf-8') as fh:
            json.dump(events, fh, ensure_ascii=False, indent=2)
    return events


In [6]:

events = load_preflop_shove_events(DB_PATH, cache_path=CACHE_PATH, force=FORCE_RELOAD)
events_df = pd.DataFrame(events)
print(f'Loaded {len(events_df)} preflop shove events with known hole cards.')
if not events_df.empty:
    display(events_df['category'].value_counts().reindex(CATEGORY_ORDER, fill_value=0))


Loaded 762 preflop shove events with known hole cards.


category
First to Bet Shove    161
3-Bet Shove           201
4-Bet Shove           207
5+ Bet Shove          193
Name: count, dtype: int64

### Starting Hand Grids

For each shove category, the 13×13 matrix shows the percentage share of
starting hands (pairs on the diagonal, suited combos upper-right).
The adjacent summary table aggregates broader hand groups.


In [7]:

SUIT_SYMBOLS = {'S', 'H', 'D', 'C'}
SUIT_NORMALIZE = {
    'S': 'S', 'H': 'H', 'D': 'D', 'C': 'C',
    's': 'S', 'h': 'H', 'd': 'D', 'c': 'C',
}
RANK_SYMBOLS = set(RANK_INDEX.keys())
SUIT_TO_CHAR = {'S': 's', 'H': 'h', 'D': 'd', 'C': 'c'}


def _parse_card(part: str) -> tuple[str, str] | None:
    part = part.strip()
    if len(part) != 2:
        return None
    first, second = part[0], part[1]
    first_up, second_up = first.upper(), second.upper()
    if first_up in SUIT_SYMBOLS and second_up in RANK_SYMBOLS:
        suit = SUIT_NORMALIZE[first]
        rank = second_up
        return rank, suit
    if second_up in SUIT_SYMBOLS and first_up in RANK_SYMBOLS:
        suit = SUIT_NORMALIZE[second]
        rank = first_up
        return rank, suit
    return None


def _cards_to_grid_position(card_string: str) -> tuple[str, str] | None:
    parts = card_string.split()
    if len(parts) != 2:
        return None
    parsed = [_parse_card(part) for part in parts]
    if any(item is None for item in parsed):
        return None
    ranks, suits = zip(*parsed)
    if ranks[0] == ranks[1]:
        return ranks[0], ranks[0]
    sorted_ranks = sorted(ranks, key=lambda r: RANK_INDEX[r])
    high, low = sorted_ranks[0], sorted_ranks[1]
    suited = suits[0] == suits[1]
    if suited:
        return high, low
    return low, high


def _classify_hand_group(card_string: str) -> tuple[str, str] | None:
    parts = card_string.split()
    if len(parts) != 2:
        return None
    parsed = [_parse_card(part) for part in parts]
    if any(item is None for item in parsed):
        return None
    ranks, suits = zip(*parsed)
    if ranks[0] == ranks[1]:
        rank = ranks[0]
        if rank == 'A':
            return 'AA', 'KK - AA'
        if rank == 'K':
            return 'KK', 'KK - AA'
        if rank == 'Q':
            return 'QQ', 'TT - QQ'
        if rank == 'J':
            return 'JJ', 'TT - QQ'
        if rank == 'T':
            return 'TT', 'TT - QQ'
        if rank in {'9', '8', '7', '6', '5', '4', '3', '2'}:
            return 'Other Pair', '22 - 99'
        if rank in BROADWAY_RANKS:
            return 'Other Broadway Pair', 'All Others'
        return 'Other Pair', 'All Others'
    sorted_ranks = sorted(ranks, key=lambda r: RANK_INDEX[r])
    high, low = sorted_ranks[0], sorted_ranks[1]
    suited = suits[0] == suits[1]
    combo_set = frozenset({high, low})
    if combo_set == frozenset({'A', 'K'}):
        return 'AK', 'AK'
    if 'A' in combo_set:
        other = next(iter(combo_set - {'A'}))
        if suited:
            if other in {'T', 'J', 'Q'}:
                return 'ATs - AQs', 'Any Other Ace'
            if other in {'2', '3', '4', '5', '6', '7', '8', '9'}:
                return 'A2s - A9s', 'Any Other Ace'
        else:
            if other in {'T', 'J', 'Q'}:
                return 'ATo - AQo', 'Any Other Ace'
            if other in {'2', '3', '4', '5', '6', '7', '8', '9'}:
                return 'A2o - A9o', 'Any Other Ace'
    if combo_set in BROADWAY_COMBOS:
        return 'Other Broadway Pair', 'All Others'
    return 'Other', 'All Others'


def build_grid(df: pd.DataFrame) -> tuple[pd.DataFrame, float]:
    grid = pd.DataFrame(0.0, index=RANKS, columns=RANKS)
    total = 0.0
    for value in df.get('hole_cards', []):
        if not isinstance(value, str):
            continue
        position = _cards_to_grid_position(value)
        if position is None:
            continue
        row, col = position
        grid.loc[row, col] += 1
        total += 1
    return grid, total


def build_summary_tables(df: pd.DataFrame) -> tuple[pd.DataFrame, pd.DataFrame, float]:
    counts1 = {group: 0 for group in HAND_GROUPS_ORDER}
    counts2 = {group: 0 for group in SUMMARY2_GROUPS_ORDER}
    total = 0
    for value in df.get('hole_cards', []):
        if not isinstance(value, str):
            continue
        labels = _classify_hand_group(value)
        if labels is None:
            continue
        label1, label2 = labels
        counts1[label1] += 1
        counts2[label2] += 1
        total += 1
    summary1 = pd.DataFrame(
        {
            'Hand Group': HAND_GROUPS_ORDER,
            'Percent': [
                (counts1[group] / total * 100.0) if total else 0.0
                for group in HAND_GROUPS_ORDER
            ],
        }
    )
    summary2 = pd.DataFrame(
        {
            'Hand Group': SUMMARY2_GROUPS_ORDER,
            'Percent': [
                (counts2[group] / total * 100.0) if total else 0.0
                for group in SUMMARY2_GROUPS_ORDER
            ],
        }
    )
    return summary1, summary2, total


def style_grid(grid: pd.DataFrame, title: str, *, fmt: str = '{:.1f}'):
    if grid.values.sum() == 0:
        return None
    styled = (
        grid.style
        .format(fmt)
        .background_gradient(cmap=BLUE_CMAP, axis=None, vmin=0)
        .set_caption(title)
    )
    return styled


def style_summary(summary_df: pd.DataFrame, title: str):
    styled = (
        summary_df.style
        .format({'Percent': '{:.1f}%'})
        .set_caption(title)
        .hide(axis='index')
        .set_properties(subset=['Hand Group'], **{'text-align': 'left'})
        .set_properties(subset=['Percent'], **{'text-align': 'right'})
    )
    return styled


def display_shove_range(category: str, *, max_bb: float | None = None, min_bb: float | None = None, title_suffix: str = ''):
    subset = events_df[events_df['category'] == category]
    if max_bb is not None:
        subset = subset[subset['bet_amount_bb'].fillna(0) <= max_bb]
    if min_bb is not None:
        subset = subset[subset['bet_amount_bb'].fillna(0) > min_bb]
    if subset.empty:
        print(f'No events recorded for {category}{title_suffix}.')
        return
    grid, total = build_grid(subset)
    if total == 0:
        print(f'No hole cards recorded for {category}{title_suffix}.')
        return
    grid_pct = grid / total * 100.0
    grid_title = f"{category}{title_suffix} - {int(total)} events"
    grid_styled = style_grid(grid_pct, grid_title)
    summary1_df, summary2_df, summary_total = build_summary_tables(subset)
    summary1_styled = style_summary(summary1_df, 'Summary')
    summary2_styled = style_summary(summary2_df, 'Summary')
    if grid_styled is None or summary1_styled is None or summary2_styled is None:
        if grid_styled is not None:
            display(grid_styled)
        else:
            print(f'No displayable data for {category}{title_suffix}.')
        return
    grid_html = grid_styled.to_html()
    summary1_html = summary1_styled.to_html()
    summary2_html = summary2_styled.to_html()
    combined_html = (
        "<div style='display:flex; gap:24px; align-items:flex-start;'>"
        f"<div>{grid_html}</div>"
        f"<div>{summary1_html}</div>"
        f"<div>{summary2_html}</div>"
        "</div>"
    )
    display(HTML(combined_html))


In [8]:

for category in CATEGORY_ORDER:
    if category == 'First to Bet Shove':
        display_shove_range(category, max_bb=30.0, title_suffix=' (≤30 BB)')
        display_shove_range(category, min_bb=30.0, title_suffix=' (>30 BB)')
    else:
        display_shove_range(category)


,A,K,Q,J,T,9,8,7,6,5,4,3,2
A,1.9,0.9,0.9,2.8,1.9,0.9,0.9,0.9,0.0,1.9,1.9,0.0,0.9
K,1.9,0.0,0.9,1.9,0.0,0.0,0.9,1.9,0.0,0.0,0.9,0.0,0.0
Q,2.8,1.9,0.9,0.9,0.0,0.0,0.0,0.0,0.0,0.9,0.0,0.0,0.0
J,4.7,0.9,0.0,3.8,0.9,0.0,0.9,0.0,0.0,0.0,0.0,0.0,0.0
T,1.9,1.9,0.9,0.9,1.9,0.0,0.9,0.0,0.0,0.0,0.0,0.9,0.0
9,4.7,0.9,0.9,0.0,0.0,3.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.9,1.9,0.0,0.0,0.9,1.9,0.0,0.0,0.0,0.0,0.0,0.0
7,2.8,0.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.9,0.0
6,0.9,0.0,0.0,0.0,0.0,0.0,0.0,1.9,1.9,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.9,0.0,0.0,0.0,0.9,0.0,0.0,1.9


,A,K,Q,J,T,9,8,7,6,5,4,3,2
A,9.1,5.5,1.8,3.6,0.0,0.0,0.0,0.0,0.0,0.0,3.6,0.0,0.0
K,0.0,1.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.8,0.0,0.0,0.0
Q,3.6,1.8,0.0,1.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.6,0.0
J,5.5,5.5,0.0,1.8,0.0,1.8,0.0,0.0,0.0,0.0,1.8,0.0,0.0
T,3.6,0.0,3.6,0.0,3.6,0.0,0.0,0.0,1.8,0.0,0.0,0.0,0.0
9,1.8,0.0,0.0,0.0,0.0,1.8,0.0,0.0,0.0,1.8,0.0,0.0,0.0
8,0.0,0.0,1.8,0.0,1.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,5.5,0.0,0.0,0.0,0.0,0.0,0.0,1.8,0.0,1.8,0.0,1.8,0.0
6,1.8,1.8,0.0,0.0,0.0,1.8,0.0,1.8,1.8,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,1.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,A,K,Q,J,T,9,8,7,6,5,4,3,2
A,3.0,1.0,0.5,0.0,0.5,1.0,0.0,0.5,0.5,0.0,2.5,0.5,1.0
K,5.5,4.5,1.0,1.0,0.0,0.5,0.0,0.0,0.5,0.5,0.0,0.5,0.0
Q,5.0,3.5,2.5,1.0,1.0,0.0,0.5,0.5,0.0,0.0,0.0,0.5,0.0
J,5.5,2.5,0.5,3.5,1.0,1.0,0.5,0.0,0.0,0.0,0.0,0.0,0.5
T,4.5,0.5,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5
9,0.5,0.0,1.5,0.0,0.0,1.5,0.0,0.0,0.0,0.0,0.0,0.5,0.0
8,1.5,0.5,0.0,0.0,0.0,0.5,2.0,0.0,0.5,0.0,0.5,0.0,0.0
7,1.0,0.5,0.5,0.0,1.0,0.0,0.0,2.0,0.5,0.0,1.0,0.0,0.0
6,1.0,0.0,0.0,0.0,0.0,0.5,0.5,0.0,3.0,0.0,0.5,0.0,0.0
5,1.0,0.0,0.5,0.0,0.5,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0


,A,K,Q,J,T,9,8,7,6,5,4,3,2
A,3.4,3.4,3.9,0.5,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.9,0.5
K,16.4,4.8,1.0,1.0,1.4,0.0,0.0,0.0,1.0,0.0,0.5,0.0,0.5
Q,3.9,1.9,5.8,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
J,3.4,0.0,0.5,4.3,0.0,1.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0
T,2.4,0.5,1.4,0.0,2.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,1.4,0.0,0.0,0.0,1.4,4.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.5,0.0,0.5,0.0,0.0,0.5,1.4,0.0,0.0,0.0,0.0,0.0,0.0
7,0.5,1.0,0.0,0.0,0.0,0.0,0.0,1.9,0.0,0.0,0.0,0.0,0.0
6,1.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,2.4,0.5,0.0,0.0,0.0
5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.9,0.0,0.0,0.0


,A,K,Q,J,T,9,8,7,6,5,4,3,2
A,8.3,7.8,3.1,0.5,2.1,0.0,0.5,0.0,0.0,1.6,0.5,0.0,0.5
K,14.0,11.4,0.0,0.5,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0
Q,1.6,2.1,8.8,0.5,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
J,1.6,0.5,0.0,3.6,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0
T,1.6,0.5,1.0,0.5,5.7,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.5,0.5,0.5,0.0,2.6,1.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1.0,0.0,0.5,0.0,0.0,0.5,1.6,0.0,0.5,0.0,0.0,0.0,0.0
7,0.5,0.0,0.0,0.0,0.0,0.0,0.5,1.6,0.0,0.0,0.0,0.0,0.0
6,0.5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.5,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.0,0.0


### Call Equity vs Shove Ranges

Run `scripts/preflop_equity.py` to precompute equity tables, then load the cached results here.


In [9]:

EQUITY_RESULTS_PATH = PROJECT_ROOT / 'analysis' / 'cache' / 'preflop_equity.json'
if EQUITY_RESULTS_PATH.exists():
    with EQUITY_RESULTS_PATH.open('r', encoding='utf-8') as fh:
        equity_results = json.load(fh)
else:
    equity_results = None
    print('Equity cache not found. Run scripts/preflop_equity.py to generate results.')
EQUITY_SCENARIOS = [
    ('first_to_bet_leq30', 'First to Bet Shove (≤30 BB)'),
    ('first_to_bet_gt30', 'First to Bet Shove (>30 BB)'),
    ('three_bet_shove', '3-Bet Shove'),
    ('four_bet_shove', '4-Bet Shove'),
    ('five_plus_bet_shove', '5+ Bet Shove'),
]
EQUITY_CMAP = LinearSegmentedColormap.from_list('equity_cmap', ['#b91c1c', '#ffffff', '#15803d'])


In [10]:
def build_equity_grid(scenario_data: dict, key: str | None = None) -> pd.DataFrame:
    grid_dict = scenario_data.get(key or 'grid', {})
    df = pd.DataFrame(index=RANKS, columns=RANKS, dtype=float)
    for row in RANKS:
        for col in RANKS:
            df.loc[row, col] = grid_dict.get(row, {}).get(col, float('nan'))
    return df


def build_equity_summaries(combo_equities: dict[str, float]) -> tuple[pd.DataFrame, pd.DataFrame]:
    aggregates1 = {group: [] for group in HAND_GROUPS_ORDER}
    aggregates2 = {group: [] for group in SUMMARY2_GROUPS_ORDER}
    for combo_str, equity in combo_equities.items():
        labels = _classify_hand_group(combo_str)
        if labels is None:
            continue
        label1, label2 = labels
        aggregates1[label1].append(equity)
        aggregates2[label2].append(equity)
    summary1 = pd.DataFrame({
        'Hand Group': HAND_GROUPS_ORDER,
        'Equity (%)': [
            (sum(values) / len(values)) if values else float('nan')
            for values in aggregates1.values()
        ],
    })
    summary2 = pd.DataFrame({
        'Hand Group': SUMMARY2_GROUPS_ORDER,
        'Equity (%)': [
            (sum(values) / len(values)) if values else float('nan')
            for values in aggregates2.values()
        ],
    })
    return summary1, summary2


def _clip(val: float | None) -> float | None:
    if val is None:
        return None
    if isinstance(val, float) and math.isnan(val):
        return None
    return float(val)


def _equity_color(val: float | None, min_red: float, max_green: float) -> str:
    val = _clip(val)
    if val is None:
        return 'background-color: #f8fafc'
    if val == 50:
        return 'background-color: #ffffff'
    top_color = (21, 128, 61)
    bottom_color = (185, 28, 28)
    white = (255, 255, 255)
    if val > 50:
        alpha = 1.0 if max_green <= 50 else (val - 50) / (max_green - 50)
        red = int(white[0] + alpha * (top_color[0] - white[0]))
        green = int(white[1] + alpha * (top_color[1] - white[1]))
        blue = int(white[2] + alpha * (top_color[2] - white[2]))
        return f'background-color: rgb({red}, {green}, {blue})'
    alpha = 1.0 if min_red >= 50 else (50 - val) / (50 - min_red)
    red = int(white[0] + alpha * (bottom_color[0] - white[0]))
    green = int(white[1] + alpha * (bottom_color[1] - white[1]))
    blue = int(white[2] + alpha * (bottom_color[2] - white[2]))
    return f'background-color: rgb({red}, {green}, {blue})'


def _ev_color(val: float | None, min_red: float, max_green: float) -> str:
    val = _clip(val)
    if val is None:
        return 'background-color: #f8fafc'
    if abs(val) < 1e-9:
        return 'background-color: #ffffff'
    top_color = (21, 128, 61)
    bottom_color = (185, 28, 28)
    white = (255, 255, 255)
    if val > 0:
        alpha = 1.0 if max_green <= 0 else val / max_green
        red = int(white[0] + alpha * (top_color[0] - white[0]))
        green = int(white[1] + alpha * (top_color[1] - white[1]))
        blue = int(white[2] + alpha * (top_color[2] - white[2]))
        return f'background-color: rgb({red}, {green}, {blue})'
    alpha = 1.0 if min_red >= 0 else (0 - val) / (0 - min_red)
    red = int(white[0] + alpha * (bottom_color[0] - white[0]))
    green = int(white[1] + alpha * (bottom_color[1] - white[1]))
    blue = int(white[2] + alpha * (bottom_color[2] - white[2]))
    return f'background-color: rgb({red}, {green}, {blue})'


def compute_ev_frame(
    equity_df: pd.DataFrame,
    *,
    call_amount: float,
    villain_amount: float | None,
    pot_before: float,
    rake_percent: float,
    rake_cap: float,
) -> pd.DataFrame:
    villain_amt = call_amount if villain_amount is None else float(villain_amount)
    final_pot = float(pot_before) + float(call_amount) + float(villain_amt)
    rake = min(float(rake_percent) * final_pot, float(rake_cap))
    post_rake_pot = final_pot - rake

    def to_ev(value: float) -> float:
        if pd.isna(value):
            return float('nan')
        return (float(value) / 100.0) * post_rake_pot - float(call_amount)

    return equity_df.map(to_ev)


def derive_ev_inputs(scenario_data: dict) -> tuple[float, float | None, float, float, float]:
    effective_stack = float(scenario_data.get('assumed_effective_stack_bb') or 0.0)
    avg_bet_total = float(scenario_data.get('avg_bet_bb') or 0.0)
    avg_bet_added = float(scenario_data.get('avg_bet_added_bb') or 0.0)

    call_amount_raw = scenario_data.get('call_amount_bb')
    call_amount = float(call_amount_raw) if call_amount_raw is not None else 0.0
    if avg_bet_added > 0:
        if call_amount <= 0:
            call_amount = avg_bet_added
        else:
            call_amount = min(call_amount, avg_bet_added)
    elif avg_bet_total > 0:
        if call_amount <= 0:
            call_amount = avg_bet_total
        else:
            call_amount = min(call_amount, avg_bet_total)
    if effective_stack > 0 and call_amount > effective_stack:
        call_amount = effective_stack

    villain_amount_raw = scenario_data.get('villain_amount_bb')
    villain_amount = float(villain_amount_raw) if villain_amount_raw is not None else None
    if villain_amount is None or villain_amount <= 0:
        base_amount = call_amount if call_amount > 0 else (
            avg_bet_added if avg_bet_added > 0 else avg_bet_total
        )
        villain_amount = base_amount if base_amount and base_amount > 0 else None

    avg_pot_before = float(scenario_data.get('avg_pot_before_bb') or 0.0)
    pot_before = float(scenario_data.get('pot_before_bb') or 0.0)
    scale_denominator = avg_bet_added if avg_bet_added > 0 else (
        avg_bet_total if avg_bet_total > 0 else None
    )
    if call_amount > 0 and scale_denominator:
        scale_factor = call_amount / scale_denominator
        if pot_before <= 0 or abs(pot_before - avg_pot_before) < 1e-6:
            pot_before = avg_pot_before * scale_factor
    elif pot_before <= 0:
        pot_before = avg_pot_before

    rake_percent = float(scenario_data.get('rake_percent', 0.05))
    rake_cap = float(scenario_data.get('rake_cap_bb', 10.0))
    return call_amount, villain_amount, pot_before, rake_percent, rake_cap


def _apply_cell_sizes(styler: pd.io.formats.style.Styler) -> pd.io.formats.style.Styler:
    cell_props = {
        'width': '30px',
        'height': '30px',
        'min-width': '30px',
        'max-width': '30px',
        'min-height': '30px',
        'max-height': '30px',
        'padding': '0px',
        'line-height': '30px',
        'text-align': 'center',
        'font-size': '9px',
        'white-space': 'nowrap',
        'overflow': 'hidden',
    }
    styler = styler.set_properties(**cell_props)
    table_styles = [
        {
            'selector': 'table',
            'props': [
                ('border-collapse', 'collapse'),
                ('table-layout', 'fixed'),
                ('width', 'auto !important'),
                ('max-width', 'none !important'),
                ('display', 'inline-block !important'),
                ('margin', '0'),
                ('border-spacing', '0'),
            ],
        },
        {
            'selector': 'thead th',
            'props': [
                ('width', '30px !important'),
                ('height', '30px !important'),
                ('min-width', '30px !important'),
                ('max-width', '30px !important'),
                ('min-height', '30px !important'),
                ('max-height', '30px !important'),
                ('box-sizing', 'border-box'),
                ('padding', '0px !important'),
                ('line-height', '30px'),
                ('text-align', 'center'),
                ('font-size', '9px'),
                ('white-space', 'nowrap'),
                ('overflow', 'hidden'),
            ],
        },
        {
            'selector': 'tbody th',
            'props': [
                ('width', '30px !important'),
                ('height', '30px !important'),
                ('min-width', '30px !important'),
                ('max-width', '30px !important'),
                ('min-height', '30px !important'),
                ('max-height', '30px !important'),
                ('box-sizing', 'border-box'),
                ('padding', '0px !important'),
                ('line-height', '30px'),
                ('text-align', 'center'),
                ('font-size', '9px'),
                ('white-space', 'nowrap'),
                ('overflow', 'hidden'),
            ],
        },
        {
            'selector': 'td',
            'props': [
                ('width', '30px !important'),
                ('height', '30px !important'),
                ('min-width', '30px !important'),
                ('max-width', '30px !important'),
                ('min-height', '30px !important'),
                ('max-height', '30px !important'),
                ('box-sizing', 'border-box'),
                ('padding', '0px !important'),
                ('line-height', '30px'),
                ('text-align', 'center'),
                ('font-size', '9px'),
                ('white-space', 'nowrap'),
                ('overflow', 'hidden'),
            ],
        },
    ]
    styler = styler.set_table_styles(table_styles, overwrite=False)
    return styler.set_table_attributes('style="border-collapse:collapse;table-layout:fixed;width:auto;display:inline-block;"')


def style_equity_grid(df: pd.DataFrame, title: str):
    values = df.values.flatten()
    finite = values[~pd.isna(values)]
    if finite.size == 0:
        return None
    gt50 = finite[finite > 50]
    lt50 = finite[finite < 50]
    max_green = float(gt50.max()) if gt50.size else 50.0
    min_red = float(lt50.min()) if lt50.size else 50.0

    def apply_color(val: float):
        return _equity_color(val, min_red, max_green)

    styled = (
        df.style
        .format('{:.1f}')
        .map(apply_color)
        .set_caption(title)
    )
    return _apply_cell_sizes(styled)


def style_ev_grid(df: pd.DataFrame, title: str):
    values = df.values.flatten()
    finite = values[~pd.isna(values)]
    if finite.size == 0:
        return None
    positives = finite[finite > 0]
    negatives = finite[finite < 0]
    max_green = float(positives.max()) if positives.size else 0.0
    min_red = float(negatives.min()) if negatives.size else 0.0

    def apply_color(val: float):
        return _ev_color(val, min_red, max_green)

    styled = (
        df.style
        .format('{:.2f}')
        .map(apply_color)
        .set_caption(title)
    )
    return _apply_cell_sizes(styled)


def display_equity_tables(scenario_key: str, scenario_label: str):
    if equity_results is None:
        return
    scenario_data = equity_results.get(scenario_key)
    if not scenario_data:
        print(f'No equity data cached for {scenario_label}.')
        return

    sections = []

    equity_df = build_equity_grid(scenario_data)
    equity_title = f"{scenario_label} Equity"
    equity_styled = style_equity_grid(equity_df, equity_title)
    if equity_styled is not None:
        sections.append(equity_styled.to_html())

    call_amount, villain_amount, pot_before, rake_percent, rake_cap = derive_ev_inputs(scenario_data)
    include_ev = scenario_key not in {'three_bet_shove', 'four_bet_shove', 'five_plus_bet_shove'}
    if include_ev and call_amount > 0 and pot_before is not None:
        ev_df = compute_ev_frame(
            equity_df,
            call_amount=call_amount,
            villain_amount=villain_amount,
            pot_before=pot_before,
            rake_percent=rake_percent,
            rake_cap=rake_cap,
        )
        ev_title = f"{scenario_label} EV (bb)"
        ev_styled = style_ev_grid(ev_df, ev_title)
        if ev_styled is not None:
            sections.append(ev_styled.to_html())

    if sections:
        container = ("<div style='display:flex;flex-direction:row;flex-wrap:wrap;gap:12px;align-items:flex-start;'>"
            + ''.join(sections)
            + "</div>")
        display(HTML(container))
        display(HTML("<hr style='margin:28px 0;border:0;border-top:1px solid #cbd5f5;'>"))


In [11]:
if equity_results is not None:
    for key, label in EQUITY_SCENARIOS:
        display_equity_tables(key, label)


,A,K,Q,J,T,9,8,7,6,5,4,3,2
A,86.0,68.7,64.0,62.6,60.1,56.4,55.8,53.1,51.9,52.1,51.3,51.0,50.4
K,67.6,79.2,56.0,54.5,52.1,49.2,47.6,46.9,45.3,44.2,43.1,43.8,42.1
Q,63.7,54.9,74.4,48.9,48.3,45.9,45.5,43.0,42.8,43.0,42.4,42.0,40.3
J,60.8,51.4,46.0,72.2,46.6,45.5,43.7,42.0,41.9,40.9,40.2,39.8,39.6
T,58.2,49.5,46.2,45.3,68.6,45.7,44.2,40.6,40.3,38.7,38.9,38.8,36.9
9,54.7,45.8,42.4,43.0,42.6,63.1,42.7,40.3,40.5,38.7,37.4,37.5,34.9
8,52.8,43.9,42.1,41.1,40.6,39.2,60.2,41.5,41.3,38.5,36.8,34.6,34.4
7,50.0,42.7,38.9,38.5,38.5,37.7,37.2,57.4,39.8,38.7,36.9,35.4,32.7
6,48.3,42.0,40.4,39.2,37.4,36.3,37.1,36.7,56.2,39.8,39.1,36.2,33.5
5,50.1,40.8,39.4,38.3,36.0,35.1,35.0,35.1,37.5,54.1,38.9,37.2,35.0


,A,K,Q,J,T,9,8,7,6,5,4,3,2
A,84.4,62.1,59.9,57.5,52.6,50.2,48.9,46.9,46.3,46.0,45.8,44.1,45.1
K,61.7,72.8,50.0,47.5,48.5,45.1,43.4,43.4,43.1,42.0,41.4,41.0,39.5
Q,58.8,45.6,70.4,46.1,44.7,43.5,42.1,40.5,40.0,40.1,38.5,37.4,38.2
J,54.3,45.2,43.8,67.8,44.6,42.0,40.6,38.2,38.1,38.6,36.6,36.2,35.7
T,49.7,44.8,43.6,41.0,62.7,42.0,41.1,39.2,39.3,37.3,36.5,35.8,35.1
9,47.2,41.7,40.5,40.0,39.8,59.2,40.7,39.2,38.2,36.1,34.0,34.1,33.7
8,45.8,40.0,38.8,38.1,38.0,37.6,55.9,38.8,38.0,36.4,34.7,32.8,33.5
7,45.3,40.8,37.1,35.4,36.9,35.6,35.9,52.6,37.3,37.0,34.1,32.3,30.7
6,42.9,38.9,37.1,34.1,34.9,34.1,35.1,35.6,51.1,37.4,34.8,33.7,33.2
5,43.4,38.2,36.2,34.9,33.4,32.8,32.2,31.9,33.0,48.7,36.2,35.4,33.3


,A,K,Q,J,T,9,8,7,6,5,4,3,2
A,84.4,61.0,57.3,54.9,52.9,48.9,48.4,47.8,45.3,47.2,45.3,44.2,42.1
K,60.4,75.3,49.6,47.0,46.1,44.9,43.1,42.8,41.9,40.3,40.7,39.4,39.2
Q,56.3,46.5,70.5,45.5,44.5,42.8,42.5,40.2,38.8,39.1,36.6,37.4,36.3
J,52.1,43.9,42.4,65.0,43.5,42.0,41.0,39.2,38.5,37.3,36.2,36.1,35.6
T,48.6,42.7,41.3,41.4,62.2,42.3,40.6,39.0,38.1,37.7,35.7,35.3,34.8
9,46.2,40.3,39.4,39.8,39.7,59.7,41.0,40.1,38.9,37.7,34.6,34.8,33.8
8,45.5,39.8,37.7,37.3,37.8,38.1,56.9,40.2,39.9,37.1,36.0,33.7,33.5
7,44.5,38.8,35.4,36.6,36.5,37.2,36.1,55.2,38.7,36.9,36.1,33.1,32.0
6,43.0,38.6,36.7,34.5,34.4,35.9,36.7,35.6,52.5,37.5,36.6,34.2,31.7
5,43.1,38.3,35.5,34.3,32.8,33.1,32.7,33.5,34.4,50.8,35.9,35.8,32.5


,A,K,Q,J,T,9,8,7,6,5,4,3,2
A,84.7,56.6,50.2,47.6,46.0,41.8,40.7,40.6,39.9,39.2,38.9,38.1,38.0
K,56.1,74.3,43.7,42.7,40.7,39.3,37.7,37.8,36.2,36.6,35.7,35.3,34.7
Q,47.5,40.6,66.1,41.3,39.7,37.8,36.9,35.2,34.9,35.3,33.8,33.5,33.1
J,43.9,38.8,38.1,59.9,40.5,37.2,36.4,35.5,34.2,33.8,33.8,32.7,32.5
T,41.7,37.2,36.9,35.8,55.3,39.2,36.5,35.7,33.6,32.3,32.1,31.4,31.1
9,39.0,34.5,34.4,34.7,34.6,52.4,36.6,35.8,34.4,32.6,30.8,30.4,30.7
8,38.7,34.0,33.3,34.5,33.0,33.0,49.2,36.8,35.1,32.8,32.6,29.9,30.2
7,37.1,33.5,31.3,32.2,31.7,33.3,32.9,48.1,35.8,34.0,33.3,30.7,29.7
6,35.6,33.5,31.2,29.4,31.3,30.9,32.1,32.4,45.3,35.2,33.3,31.4,30.3
5,36.9,32.2,30.5,30.3,28.7,28.9,30.4,31.3,30.8,44.9,34.7,32.8,30.9


,A,K,Q,J,T,9,8,7,6,5,4,3,2
A,83.6,52.9,45.8,43.5,40.6,37.6,37.7,37.8,35.9,37.3,36.7,36.5,35.7
K,50.6,69.0,39.6,36.9,36.3,33.6,33.0,32.2,33.5,33.2,31.6,32.0,31.4
Q,42.2,35.0,55.5,36.5,36.0,34.4,33.7,31.7,31.0,31.6,29.9,30.5,30.1
J,39.3,34.0,34.0,52.1,35.3,33.1,32.4,31.7,30.9,30.6,30.2,29.3,29.4
T,37.1,32.6,32.8,31.5,47.9,34.7,32.6,32.6,31.1,28.5,29.3,28.8,27.9
9,35.1,31.0,30.6,30.3,30.2,43.7,32.1,32.1,30.6,30.1,28.4,28.1,28.1
8,33.7,30.1,28.5,29.3,28.7,28.6,41.9,32.9,31.6,29.7,28.6,26.9,27.4
7,33.3,29.1,28.5,28.1,28.6,28.2,28.9,41.0,32.4,31.1,30.2,29.1,26.6
6,33.1,28.4,27.4,26.9,26.6,28.1,28.0,29.1,40.4,32.1,30.4,29.3,28.0
5,33.7,29.3,26.8,26.8,25.5,25.5,26.4,26.8,29.5,39.4,32.5,31.0,28.3
